# How to reproduce refcat loading 

This was run at the USDF. Check to see if you get the same answer with the summit butler

In [1]:
from lsst.daf.butler import Butler
import lsst.daf.butler as dafButler
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig
from lsst.pipe.tasks.calibrate import CalibrateTask, CalibrateConfig
from lsst.utils import getPackageDir
from lsst.meas.algorithms import ReferenceObjectLoader
from lsst.daf.base import DateTime
import astropy.time

In [2]:
#butler = Butler('/repo/embargo', collections=['LATISS/runs/quickLook',])
import lsst.summit.utils.butlerUtils as butlerUtils
butler = butlerUtils.makeDefaultLatissButler(embargo=True)


# Setting up tasks isn't necessary, but just to have available:
config = CharacterizeImageConfig()
basicConfig = CharacterizeImageConfig()
obs_lsst = getPackageDir("obs_lsst")
config.load(os.path.join(obs_lsst, "config", "characterizeImage.py"))
config.load(os.path.join(obs_lsst, "config", "latiss", "characterizeImage.py"))
config.measurement = basicConfig.measurement
charImage = CharacterizeImageTask(config=config)

config = CalibrateConfig()
basicConfig = CalibrateConfig()
config.load(os.path.join(obs_lsst, "config", "calibrate.py"))
config.load(os.path.join(obs_lsst, "config", "latiss", "calibrate.py"))
config.measurement = basicConfig.measurement

calibrate = CalibrateTask(config=config, icSourceSchema=charImage.schema)

# Preload refcat dataids and deferred dataset handles
astrometryRefCatName = calibrate.config.connections.astromRefCat
astromRefs = butler.registry.queryDatasets(astrometryRefCatName).expanded()
astromHandles = [dafButler.DeferredDatasetHandle(butler=butler, ref=ref, parameters=None)
                 for ref in astromRefs]
astromDataIds = [butler.registry.expandDataId(ref.dataId) for ref in astromRefs]

loader = ReferenceObjectLoader(
    astromDataIds,
    astromHandles,
    name=astrometryRefCatName,
    config=calibrate.config.astromRefObjLoader
)


In [3]:
dataId = {'day_obs': 20221207, 'seq_num': 56, 'detector':0}
exposure = butler.get("quickLookExp", collections=['LATISS/runs/quickLook',], dataId=dataId)

In [4]:
bbox=exposure.getBBox()
wcs=exposure.info.getWcs()
photoCalib=exposure.info.getPhotoCalib()
filterLabel = exposure.info.getFilter()
filterName = filterLabel.bandLabel if filterLabel is not None else None

if exposure.info.hasVisitInfo():
    epochTaiMjd = exposure.visitInfo.date.get(system=DateTime.MJD, scale=DateTime.TAI)
    epoch = astropy.time.Time(epochTaiMjd, scale="tai", format="mjd")

In [5]:
loadRes = loader.loadPixelBox(
            bbox=bbox,
            wcs=wcs,
            filterName=filterName,
            epoch=epoch,
        )

In [6]:
loadRes

Struct(refCat=         id               coord_ra      ...   slot_Centroid_y   hasCentroid
                            rad         ...                                
------------------- ------------------- ... ------------------- -----------
4685951397518077696 0.21943853968556495 ...  -369.9058228838078        True
4685951397518077824  0.2194402462256785 ...   75.11871528214829        True
4685951363158578432 0.21864208890609196 ...  -603.4876931602446        True
4685951367422201344 0.21855306734170216 ... -329.64206866411064        True
4685951298743193984 0.21843114042009543 ...  -877.1016505325495        True
4685951367462670848 0.21805295062245617 ...  -843.9187392264564        True
4685951363158432384 0.21872518343778627 ...  -706.6455688091103        True
4685951401781940096 0.21916847299592437 ...  -413.1528360388363        True
4685951367462670976 0.21786510618408791 ...  -799.4586364516042        True
4685951397518074240  0.2191903444087068 ...  -357.0489429445945        Tru

In [7]:
len(loadRes.refCat)

22821

In [8]:
loadRes.refCat[0]

<class 'lsst.afw.table.SimpleRecord'>
id: 4685951397518077696
coord_ra: 0.219439 rad
coord_dec: -1.27334 rad
phot_g_mean_flux: 80255.8
phot_bp_mean_flux: 94605.4
phot_rp_mean_flux: 192367
phot_g_mean_fluxErr: 302.364
phot_bp_mean_fluxErr: 5834.91
phot_rp_mean_fluxErr: 33157.9
coord_raErr: 2.25264e-08
coord_decErr: 1.53864e-08
epoch: 57205.9
pm_ra: 6.21106e-09 rad
pm_dec: -3.30518e-09 rad
pm_raErr: 3.02474e-09
pm_decErr: 2.06476e-09
pm_flag: 0
parallax: 3.31542e-09 rad
parallaxErr: 1.28006e-09 rad
parallax_flag: 0
astrometric_excess_noise: 1.34285
phot_variable_flag: NOT_AVAILABLE
centroid_x: 925.256
centroid_y: -369.906
hasCentroid: 1

In [ ]:
loadRes.keys()